In [0]:
# Cell 0: Imports
import os
import json
import base64
import uuid
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, TimestampType
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    save_with_audit,
    get_codelist_flexible_mapper,
    find_best_header_row  # Generic robust header detection with config-driven required columns
)

# Note: This notebook uses mapInPandas for scalable parallel processing of codelist sheets
# Required columns are loaded from config: validation.codelists.required_columns


In [ ]:
# Cell 1: Check Upstream Status & Documents Found
# Check if extract_excel_sheets found documents
# Note: Codelists are now processed BEFORE transfer variables (they provide lookup data)

print("Checking upstream task status (extract_excel_sheets)...")
try:
    upstream_status = dbutils.jobs.taskValues.get(
        taskKey="extract_excel_sheets", 
        key="excel_extraction_status"
    )
    print(f"  Upstream status (extract_excel_sheets): {upstream_status}")
    
    if upstream_status == "FAILED":
        print("\n⚠️  Upstream task (extract_excel_sheets) FAILED")
        print("   Exiting...")
        dbutils.notebook.exit("SKIPPED - Upstream task failed")
    elif upstream_status == "NO_DOCUMENTS":
        print("\n⚠️  No documents to process")
        dbutils.notebook.exit("SKIPPED - No documents found")
    elif upstream_status == "SUCCESS":
        doc_count = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="excel_sheets_count")
        print(f"  Upstream extracted: {doc_count} sheets")
    
except Exception as e:
    print(f"⚠ Could not read upstream status: {e}")
    print("  Continuing anyway (backwards compatibility)")

# Check if documents were found
try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("\n⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping codelist processing.")
        dbutils.notebook.exit("SKIPPED - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass

print("\n✓ All upstream checks passed. Proceeding with codelist processing...")

In [0]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

# Get validation config for codelists (from config: validation.codelists)
validation_config = pipeline_config.get('validation', {})
codelist_validation = validation_config.get('codelists', {})
codelist_required_columns = codelist_validation.get('required_columns', ['codelist_reference', 'code_value'])
# Header patterns to find the header row - ALL patterns must match in the same row
codelist_header_patterns = codelist_validation.get('header_patterns', ['codelist.*ref|reference', 'values?'])

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Codelist required columns: {codelist_required_columns}")
print(f"Codelist header patterns: {codelist_header_patterns}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")

In [0]:
# Cell 3: Construct Table Names
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
history_table = f"{catalog}.{bronze_schema}.md_dta_history"
status_table = f"{catalog}.{bronze_schema}.md_document_status"
output_table = f"{catalog}.{silver_schema}.md_codelists_normalized"

print(f"Sheets table: {sheets_table}")
print(f"History table: {history_table}")
print(f"Status table: {status_table}")
print(f"Output table: {output_table}")


In [0]:
# Cell 4: Read Codelist Sheets
# Filter for sheets categorized as 'codelists'

print(f"{'='*80}")
print(f"Reading codelist sheets...")
print(f"{'='*80}")

df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "codelists"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

# Join with parent documents to get content_base64
# Only join with current versions (defensive filter - sheets should already be from current versions)
df_with_content = df_sheets.join(
    spark.table(history_table).filter(
        F.col("is_current") == True  # Only current versions
    ).select(
        F.col("document_id").alias("parent_id"),
        "content_base64"
    ),
    df_sheets.parent_document_id == F.col("parent_id")
)

sheet_count = df_with_content.count()
print(f"Found {sheet_count} codelist sheets to process")
print(f"{'='*80}")

if sheet_count == 0:
    print("No codelist sheets to process. Exiting.")
    dbutils.notebook.exit("No codelist sheets to process")

In [0]:
# Cell 5: Process Codelist Sheets Using mapInPandas
# Scalable parallel processing across Spark workers
# NOTE: Codelists are OPTIONAL - empty sheets or sheets with only headers are valid

print(f"{'='*80}")
print(f"Processing codelist sheets with mapInPandas...")
print(f"{'='*80}")

# Define result schema for codelist processing
codelist_result_schema = StructType([
    StructField("codelist_id", StringType(), False),
    StructField("parent_document_id", StringType(), True),
    StructField("trial_id", StringType(), False),
    StructField("data_stream_type", StringType(), False),
    StructField("data_provider_name", StringType(), False),
    StructField("transfer_variable_name", StringType(), False),
    StructField("values", ArrayType(StringType()), True),
    StructField("value_sdtm_mapping", ArrayType(
        StructType([
            StructField("value", StringType(), True),
            StructField("sdtm_value", StringType(), True)
        ])
    ), True),
    StructField("row_status", StringType(), True),
    StructField("categorization_notes", StringType(), True),
    StructField("processing_status", StringType(), True)  # SUCCESS or ERROR (empty sheets are skipped)
])

def process_codelist_pandas(pdf_iterator):
    """
    Process codelist sheets using mapInPandas.
    Each partition processes sheets in parallel across Spark workers.
    
    NOTE: Codelists are optional - empty sheets are valid (no controlled terminology).
    """
    for pdf in pdf_iterator:
        results = []
        
        for _, row in pdf.iterrows():
            sheet_name = row['sheet_name']
            source_path = row['source_file_path']
            content_b64 = row['content_base64']
            parent_document_id = row['parent_document_id']
            
            trial_id = row['trial_id']
            data_stream_type = row['data_stream_type']
            data_provider_name = row['data_provider_name']
            
            try:
                # Decode Excel from base64
                excel_bytes = base64.b64decode(content_b64)
                file_ext = os.path.splitext(source_path.lower())[1]
                engine = 'xlrd' if file_ext in ['.xls', '.xlt'] else 'openpyxl'
                
                # Get config from columns (passed via JSON)
                required_cols = json.loads(row['required_columns_json'])
                header_patterns = json.loads(row['header_patterns_json'])
                
                # Use generic robust header detection with config-driven patterns
                # First scans for row matching ALL header_patterns (e.g., "codelist.*ref", "values?")
                # Then uses column mapper to extract standardized column names
                df_sheet, best_header_row, col_map, missing_required = find_best_header_row(
                    excel_bytes=excel_bytes,
                    sheet_name=sheet_name,
                    column_mapper_func=get_codelist_flexible_mapper,
                    required_columns=required_cols,
                    header_patterns=header_patterns,  # Patterns to find header row
                    max_try_rows=15,  # Handle instruction rows that may precede headers
                    engine=engine,
                    debug=True
                )
                
                actual_columns = list(df_sheet.columns)
                has_sdtm = 'sdtm_value' in col_map.values()
                
                # Check for missing required columns
                if missing_required:
                    # If we can't find the required columns, this sheet has no valid codelist structure
                    # Simply skip - don't add any record (codelists are optional)
                    if len(df_sheet) == 0 or len(actual_columns) == 0:
                        print(f"  ℹ️  Sheet '{sheet_name}' is empty - skipping (codelists are optional)")
                    else:
                        print(f"  ℹ️  Sheet '{sheet_name}' missing required columns: {missing_required}")
                        print(f"      Found columns: {actual_columns[:5]}")
                        print(f"      Skipping - no valid codelist structure (codelists are optional)")
                    continue
                
                # Rename columns using the mapping
                df_sheet = df_sheet.rename(columns=col_map)
                print(f"  ✓ Column mapping applied: {col_map}")
                
                # Filter rows with codelist_reference (value can be optional for some rows)
                if 'codelist_reference' not in df_sheet.columns:
                    print(f"  ⚠️  Sheet '{sheet_name}' - codelist_reference column not in DataFrame after mapping")
                    results.append({
                        'codelist_id': str(uuid.uuid4()),
                        'parent_document_id': parent_document_id,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'transfer_variable_name': f"SHEET:{sheet_name}",
                        'values': [],
                        'value_sdtm_mapping': [],
                        'row_status': 'MANUAL_REVIEW_REQUIRED',
                        'categorization_notes': f"codelist_reference column missing after mapping. Columns: {list(df_sheet.columns)}",
                        'processing_status': 'ERROR'
                    })
                    continue
                
                # Filter to rows with at least a reference
                df_sheet = df_sheet[df_sheet['codelist_reference'].notna()].copy()
                
                # Check if sheet has any data rows after filtering
                if len(df_sheet) == 0:
                    # Sheet has headers but no data rows - skip (don't add any record)
                    print(f"  ℹ️  Sheet '{sheet_name}' has headers but no data rows - skipping (no codelists)")
                    continue
                
                # Group by codelist_reference and collect values with mappings
                codelists_found = 0
                for var_name, grp in df_sheet.groupby('codelist_reference'):
                    var_name_clean = str(var_name).strip()
                    if not var_name_clean or var_name_clean.lower() == 'nan':
                        continue
                    
                    values_set = set()
                    value_sdtm_mappings = []
                    
                    # Collect value-sdtm pairs
                    for _, r in grp.iterrows():
                        code_val = r.get('code_value')
                        if pd.notna(code_val):
                            value = str(code_val).strip()
                            if value and value.lower() != 'nan':
                                values_set.add(value)
                                
                                sdtm = None
                                if has_sdtm and 'sdtm_value' in grp.columns:
                                    sdtm_raw = r.get('sdtm_value')
                                    if pd.notna(sdtm_raw):
                                        sdtm = str(sdtm_raw).strip()
                                        if sdtm.lower() == 'nan' or sdtm == '':
                                            sdtm = None
                                
                                value_sdtm_mappings.append({
                                    'value': value,
                                    'sdtm_value': sdtm
                                })
                    
                    # Sort and deduplicate
                    values_list = sorted(list(values_set))
                    unique_mappings = {}
                    for m in value_sdtm_mappings:
                        if m['value'] not in unique_mappings:
                            unique_mappings[m['value']] = m
                    value_sdtm_list = sorted(list(unique_mappings.values()), key=lambda x: x['value'])
                    
                    codelists_found += 1
                    results.append({
                        'codelist_id': str(uuid.uuid4()),
                        'parent_document_id': parent_document_id,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'transfer_variable_name': var_name_clean,
                        'values': values_list,
                        'value_sdtm_mapping': value_sdtm_list,
                        'row_status': 'COMPLETED',
                        'categorization_notes': None,
                        'processing_status': 'SUCCESS'
                    })
                
                print(f"  ✓ Sheet '{sheet_name}' processed: {codelists_found} codelists found")
                
            except Exception as e:
                # Error handling - record the error
                print(f"  ✗ Error processing sheet '{sheet_name}': {str(e)[:200]}")
                results.append({
                    'codelist_id': str(uuid.uuid4()),
                    'parent_document_id': parent_document_id,
                    'trial_id': trial_id,
                    'data_stream_type': data_stream_type,
                    'data_provider_name': data_provider_name,
                    'transfer_variable_name': f"SHEET:{sheet_name}",
                    'values': [],
                    'value_sdtm_mapping': [],
                    'row_status': 'MANUAL_REVIEW_REQUIRED',
                    'categorization_notes': f"Error processing sheet: {str(e)[:500]}",
                    'processing_status': 'ERROR'
                })
        
        # Yield results
        if results:
            yield pd.DataFrame(results)
        else:
            yield pd.DataFrame(columns=[f.name for f in codelist_result_schema.fields])

# Serialize config to JSON for passing to mapInPandas workers
required_columns_json = json.dumps(codelist_required_columns)
header_patterns_json = json.dumps(codelist_header_patterns)
print(f"Required columns (from config): {codelist_required_columns}")
print(f"Header patterns (from config): {codelist_header_patterns}")

# Add config as columns to input DataFrame for distribution to workers
df_with_config = df_with_content \
    .withColumn("required_columns_json", lit(required_columns_json)) \
    .withColumn("header_patterns_json", lit(header_patterns_json))

# Repartition for optimal parallelism
num_partitions = min(max(1, sheet_count // 10), 200)
df_repartitioned = df_with_config.repartition(num_partitions)

print(f"Using {num_partitions} partitions for parallel processing")

# Execute mapInPandas
df_codelist_results = df_repartitioned.mapInPandas(process_codelist_pandas, codelist_result_schema)

# Count results (EMPTY sheets are skipped, not recorded)
total_count = df_codelist_results.count()
success_count = df_codelist_results.filter(F.col("processing_status") == "SUCCESS").count()
error_count = df_codelist_results.filter(F.col("processing_status") == "ERROR").count()

print(f"\n{'='*80}")
print(f"Processing Results:")
print(f"  Total codelist records: {total_count}")
print(f"  SUCCESS: {success_count}")
print(f"  ERROR: {error_count}")
print(f"  (Empty sheets are skipped - codelists are optional)")
print(f"{'='*80}")

# Prepare for output - remove processing_status column
all_codelist_records = df_codelist_results.drop("processing_status")

# Handle case where no codelists found - this is VALID (codelists are optional)
if total_count == 0:
    print("ℹ️  No codelist sheets found - this is valid (codelists are optional)")
    dbutils.jobs.taskValues.set(key="codelist_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="codelist_success_count", value="0")
    dbutils.jobs.taskValues.set(key="codelist_error_count", value="0")
    dbutils.notebook.exit("No codelist sheets - codelists are optional")


In [0]:
# Cell 6: Write Results to Silver Tables & Set Task Values
# Write with audit columns and signal success/failure
# NOTE: Codelists are OPTIONAL - empty codelists are valid

print(f"\n{'='*80}")
print(f"Writing Results to Silver Table...")
print(f"{'='*80}")

# Display sample
print(f"\nSample codelist records:")
all_codelist_records.show(5, truncate=False)

# Construct output table name
output_table = f"{catalog}.{silver_schema}.md_codelists_normalized"

# Write to silver table with audit columns using APPEND mode for full lineage
# Deduplication happens at the gold layer
print(f"\nWriting {total_count} records to {output_table}...")
save_with_audit(
    df=all_codelist_records,
    table_name=output_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode='append'  # Use append for full lineage (dedup at gold layer)
)

print(f"\n{'='*80}")
print(f"✓ Successfully wrote {total_count} codelist records")
print(f"  Table: {output_table}")
print(f"  Mode: append (full lineage, dedup at gold layer)")
print(f"{'='*80}")

# Set task values for monitoring
# NOTE: Codelists are OPTIONAL - empty sheets are skipped (not recorded)
if success_count == 0 and error_count > 0:
    # All records failed - set status to MANUAL_REVIEW but don't raise exception
    # Codelists are optional, so we warn but continue
    task_status = "MANUAL_REVIEW_REQUIRED"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="codelist_error_count", value=str(error_count))
    print(f"\n⚠️  Warning: {error_count} codelist records need review")
    print(f"   Codelists are optional - continuing workflow")
elif success_count > 0:
    # Some records processed successfully
    task_status = "SUCCESS"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="codelist_error_count", value=str(error_count))
    print(f"\n✓ Task status: {task_status}")
    print(f"  Codelists found: {success_count}")
    if error_count > 0:
        print(f"  ⚠️  {error_count} records need review")
else:
    # No codelists found - this is valid (codelists are optional)
    task_status = "NO_DATA"
    dbutils.jobs.taskValues.set(key="codelist_status", value=task_status)
    dbutils.jobs.taskValues.set(key="codelist_success_count", value="0")
    dbutils.jobs.taskValues.set(key="codelist_error_count", value="0")
    print(f"\n✓ No codelists found - this is valid (codelists are optional)")

In [ ]:
# Cell 7: Codelist Processing Complete
# Note: Transfer variables will look up codelists directly when they are processed
# (Codelists are processed FIRST, before transfer variables)
# This approach ensures silver tables are append-only with no updates

print(f"{'='*80}")
print(f"✓ Codelist processing complete")
print(f"  Codelists written to: {output_table}")
print(f"  Transfer variables will look up codelist values when processed")
print(f"{'='*80}")


In [0]:
# Cell 7: Verify Results

print(f"\nVerifying table: {output_table}")
result_count = spark.table(output_table).count()
print(f"Total codelists: {result_count}")

print(f"\nSample codelists:")
spark.table(output_table).select(
    "codelist_id",
    "trial_id",
    "transfer_variable_name",
    "values",
    "value_sdtm_mapping",
    "databricks_job_name"
).show(5, truncate=False)

# Show exploded mappings for verification
print(f"\nExample: Exploded value->SDTM mappings:")
spark.table(output_table).select(
    "transfer_variable_name",
    F.explode("value_sdtm_mapping").alias("mapping")
).select(
    "transfer_variable_name",
    "mapping.value",
    "mapping.sdtm_value"
).limit(10).show(truncate=False)
